# Train an ALPaCA dynamics model

Trains and saves an ALPaCA dynamics model on Rocker-bogie dynamics

In [1]:
import sys
sys.path.append('../')

In [2]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import matplotlib.pyplot as plt

# import dynamics (can replace with a different env if desired)
from src.envs.rocker_bogie_with_terrain_simplified import RockerBogieDynamics as Dynamics
from src.envs.rocker_bogie_with_terrain_simplified import state_space, action_space

# import dataloading tools
from src.models.dataset import TorchDatasetWrapper, DynamicsRandomDataset
from torch.utils.data import DataLoader

# import modeling tools
from src.models import Alpaca, AlpacaDynamics

### Define where to save the model

In [3]:
MODEL_PATH = "trained_models/rocker_bogie_alp_alone"

Initialize dynamics to train on

In [4]:
dynamics = Dynamics()

c rand
phi rand


In [5]:
config = {
    'x_dim': dynamics.ob_dim,
    'u_dim': action_space.dimension,
    'learning_rate': 5e-5,
    'sigma_eps': [1e-2]*dynamics.ob_dim,
    'data_horizon': 40,
    'max_context': 10,
    'multistep_training': False,
    'learning_rate_decay': False,
    'phi_dim': 64,
    'learnable_noise':True,
}

model = Alpaca(config)
dynmodel = AlpacaDynamics(model)

In [6]:
dataset = TorchDatasetWrapper(DynamicsRandomDataset(dynamics, state_space, action_space))
dataloader = DataLoader(dataset, batch_size=20)

Training progress is logged in the directory `runs` for visualization in Tensorboard

In [7]:
dynmodel.train(dataloader, 3, val_dataloader=None, verbose=True)
# dynmodel.train(dataloader, 3000, val_dataloader=None, verbose=True)

 33%|███▎      | 1/3 [00:00<00:01,  1.33it/s, tr_loss=35.6763]
Other loss =  35.6763  Ortho loss weighted= 0.0 Bounds loss weighted= 0.0
 67%|██████▋   | 2/3 [00:01<00:00,  1.34it/s, tr_loss=30.733265]
Other loss =  30.733265  Ortho loss weighted= 0.0 Bounds loss weighted= 0.0
100%|██████████| 3/3 [00:02<00:00,  1.39it/s, tr_loss=26.658516]
Other loss =  26.658516  Ortho loss weighted= 0.0 Bounds loss weighted= 0.0



In [8]:
dynmodel.save(MODEL_PATH)